<a href="https://colab.research.google.com/github/jobian-ai/Hello-World/blob/main/fakerTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install dependent modules

In [ ]:
pip install Faker


### Import modules

In [ ]:
from faker import Faker
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import random
import os


---
# Connect to the database




In [ ]:
dbconn = sqlite3.connect('FakeHR.db') # permanent database
print(dbconn) # Check the connection to the database.

In [ ]:
cursor = dbconn.cursor()
cursor

# Working with Faker

In [ ]:
# initialize a generator
fake = Faker()

### Born after starting at job - Problem.

Issue with having Faker generate fake Hire Dates (OBD's) and Date of Births (DOB's) is that they aren't realistic.  Could be born after the Hire Date.  Hence, I used the approach (found here)[https://www.tonic.ai/blog/how-to-generate-simple-test-data-with-faker]

Defining a function that links the DOB and OBD's

In [ ]:
def birth_and_start_date(x):
   sd = fake.date_between(start_date="-30y", end_date="now")
   delta = timedelta(days=365*random.randint(22,52))
   bd = sd-delta

   return {'DOB':bd, 'OBD': sd}

___
# Start 

## Creating the faker file here
___

### Generating for the gents. . .

In [ ]:
# 'LINKED_DATES':birth_and_start_date.(x=[0])}
fake_gents = [
{'MALE_FN': fake.first_name_male(), 'MALE_MN': fake.first_name_male(), 'MALE_LN': fake.last_name_male(),
'SSN':fake.ssn()}
for _ in range(500)]

In [ ]:
df = pd.DataFrame(fake_gents)
df.head()

### Coverting to a dataframe

In [ ]:
df = pd.DataFrame(fake_gents)

### Appending the DOB and OBD date columns

(Multiple column help)[https://www.pauldesalvo.com/how-to-return-multiple-columns-from-pandas-using-the-apply-function/]

In [ ]:
df[['DOB', 'OBD']] = df.apply(birth_and_start_date, axis=1, result_type ='expand')

In [ ]:
df.head()

### Creating a Full Name field

In [ ]:
df['FULL_NAME'] = df['MALE_LN'] + (', ' + df['MALE_FN'])  + (' '+ df['MALE_MN'].str[0])

### Creating the email address based on first & last names

In [ ]:
df['EMAIL'] = df['MALE_FN'].str[0] + (df['MALE_LN'] + ('@FEDERAL.GOV'))
df['EMAIL'] = df['EMAIL'].str.lower()

### Adding a column for Sex.

In [ ]:
df['SEX'] = ('M')

### Converting the datetime to dates

I like to see just the dates.

I'm also apply a sort here.

In [ ]:
df.sort_values(by=['OBD'], inplace=True)

In [ ]:
df['OBD']= pd.to_datetime(df['OBD']) # This line converts the string 'date' to a real datetime
df['OBD']= df['OBD'].dt.date # This line strips the time data off and leave just a date
df['DOB']= pd.to_datetime(df['DOB']) # This line converts the string 'date' to a real datetime
df['DOB']= df['DOB'].dt.date # This line strips the time data off and leave just a date
df.head(5)

### Testing for uniquness of Email / SSN

In [ ]:
df['EMAIL'].is_unique 
# df['SSN'].is_unique 

___

## Creating the Female "Workers"

In [ ]:
fake_ladies = [
{'FEMALE_FN': fake.first_name_female(), 'FEMALE_MN': fake.first_name_female(), 'FEMALE_LN': fake.last_name_female(),
'SSN':fake.ssn()}
for _ in range(500)]

In [ ]:
df2 = pd.DataFrame(fake_ladies)

In [ ]:
df2[['DOB', 'OBD']] = df2.apply(birth_and_start_date, axis=1, result_type ='expand')

In [ ]:
df2['FULL_NAME'] = df2['FEMALE_LN'] + (', ' + df2['FEMALE_FN'])  + (' '+ df2['FEMALE_MN'].str[0])
df2['EMAIL'] = df2['FEMALE_FN'].str[0] + (df2['FEMALE_LN'] + ('@FEDERAL.GOV'))
df2['EMAIL'] = df2['EMAIL'].str.lower()
df2['SEX'] = ('F')


In [ ]:
df2.sort_values(by=['OBD'], inplace=True)

In [ ]:
df2['OBD']= pd.to_datetime(df2['OBD']) # This line converts the string 'date' to a real datetime
df2['OBD']= df2['OBD'].dt.date # This line strips the time data off and leave just a date
df2['DOB']= pd.to_datetime(df2['DOB']) # This line converts the string 'date' to a real datetime
df2['DOB']= df2['DOB'].dt.date # This line strips the time data off and leave just a date
df2.head(5)

### Renaming the columns so the injest does not fail.

In [ ]:
df = df.rename(columns={"MALE_FN":"FNAME", "MALE_MN":"MNAME","MALE_LN":"LNAME"})
df2 = df2.rename(columns={"FEMALE_FN":"FNAME", "FEMALE_MN":"MNAME","FEMALE_LN":"LNAME"})

In [ ]:
frames = [df, df2]

In [ ]:
df3 = pd.concat(frames)
df3.sort_values(by=['OBD'], inplace=True)
df3.head(5)

In [ ]:
df3.dtypes

In [ ]:
df3.shape

### Dropping the duplicate email addresses. We will do this again at the end.





In [ ]:
df3 = df3.drop_duplicates(subset='EMAIL', keep="first")
df3.shape

### Time calculations

I wanted to check how old and how long people worked there.

Had to covert back to datetime first!!

In [ ]:
df3['DOB']= pd.to_datetime(df3['DOB'])
df3['OBD']= pd.to_datetime(df3['OBD'])


In [ ]:
current_datetime = datetime.now()
dt = current_datetime.date()
dt_tomorrow = dt + timedelta(days=1)
dt

### A function.

In [ ]:
def f(end):
    # r = relativedelta(pd.to_datetime('now'), end) 
    r = relativedelta(pd.to_datetime(dt_tomorrow), end) 
    # return '{} years' .format(r.years)  # This line adds the word "years" into the field.
    return '{}' .format(r.years)

In [ ]:
df3['AGE'] = df3["DOB"].apply(f)

In [ ]:
df3['SERVICE_TIME'] = df3["OBD"].apply(f)

In [ ]:
df3.head(10)

### Getting back to just Dates.

Not datetime.

In [ ]:
df3['OBD']= pd.to_datetime(df3['OBD']) # This line converts the string 'date' to a real datetime
df3['OBD']= df3['OBD'].dt.date # This line strips the time data off and leave just a date
df3['DOB']= pd.to_datetime(df3['DOB']) # This line converts the string 'date' to a real datetime
df3['DOB']= df3['DOB'].dt.date # This line strips the time data off and leave just a date

In [ ]:
df3.head(25)

### Put the data in the database.

In [ ]:
df3.to_sql('tmp_FAKER', dbconn, index=0, if_exists='replace')

At this point I want to save the data to the Colab instance in the folder 'DATA' which I am creating manually.

In [43]:
# outfile = '/content/DATA/faker-02.csv # <- for online saving to G drive
outfile = r"C:\Users\Chuck\Documents\PROJECTS\Hello-World\FakeHR\DATA\test.csv"
df3.to_csv(outfile, header = True, index=False, encoding = 'utf-8-sig')

## HOLD UP!
___
Here I go back up to 'Start' (see TOC) and create another faker file and save it in the DATA folder.  Remember to rename the file eg. faker-01.csv to faker-02.csv

NOTE: I manualy created the 'DATA' folder
___

**TO CHATGPT:** <Write a Python script using pandas to combine data from multiple Excel spreadsheets located in the "DATA" folder.  The spreadsheets have column for "FNAME", "MNAME", "LNAME", "SSN", "DOB", "OBD", "EMAIL", "SEX", and "LOCATION".>

In [ ]:
# Define the directory path where the CSV files are located
directory = '/content/DATA/'

# Initialize an empty DataFrame to store the combined data
combined_data = pd.DataFrame()

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path)
        df['Source'] = filename  # Add 'Source' column with the filename
        combined_data = combined_data.append(df, ignore_index=True)

# Display the combined data
print(combined_data)

Dropping these columns while I'm working with ChatGPT.

In [ ]:
combined_data.drop(['AGE', 'SERVICE_TIME'], axis=1, inplace=True)

**TO CHAT GPT: **<The "DOB" column is a date in the format YYYY-MM-DD.  Add a column to the dataframe that calculates age to nearest year>

In [ ]:
# Calculate age based on 'DOB' column
combined_data['DOB'] = pd.to_datetime(combined_data['DOB'])  # Convert 'DOB' column to datetime
combined_data['CALCULATED_AGE'] = (datetime.now() - combined_data['DOB']) // pd.Timedelta(days=365)  # Calculate age

# Display the combined data with calculated age
print(combined_data)

In [ ]:
# combined_data.drop(['CALCULATED_OBD_AGE'], axis=1, inplace=True)

In [ ]:
# Calculate SERVICE TIME based on 'OBD' column
combined_data['OBD'] = pd.to_datetime(combined_data['OBD'])  # Convert 'OBD' column to datetime
combined_data['SERVICE_TIME'] = (datetime.now() - combined_data['OBD']) // pd.Timedelta(days=365)  # Calculate age

# Display the combined data with calculated OBD age
print(combined_data)

I want the "SOURCE" column to be the last coumn on the right

In [ ]:
# Reorder columns and move 'Source' to the last column
cols = list(combined_data.columns)
cols.remove('Source')  # Remove 'Source' from current position
cols.append('Source')  # Add 'Source' at the end
combined_data = combined_data[cols]

## Check for the uniquness of the EMAIL address.

In [ ]:
combined_data['EMAIL'].is_unique 


Check for the uniquness of the SSN.

In [ ]:
combined_data['SSN'].is_unique 

## DELETE any people with duplicate EMAIL addresses. RIP!

In [ ]:
combined_data = combined_data.drop_duplicates(subset='EMAIL', keep="first")
combined_data.shape

### Drop and re-create the table in the database.

In [ ]:
cursor = dbconn.cursor()
cursor.execute('''DROP TABLE IF EXISTS tmp_FAKER''')
dbconn.commit()

### Put the data in the database.

In [ ]:
combined_data.to_sql('tmp_FAKER', dbconn, index=0, if_exists='replace')